In [1]:
import numpy as np
import pandas as pd
import csv
#import torch
from math import factorial
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [2]:
def pos(i,prod):
    '''
    Compute positive function and gradient information
    
    input:
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fpt - positive function value
        gfpt - positive function gradient
    '''
    fpt = 0.0 
    gfpt = 0.0 
    fpt = (L/2+prod)**i 
    gfpt = i*(L/2+prod)**(i-1) # no xt yet!
    return fpt,gfpt               

In [3]:
def comb(n, k):
    '''
    Compute combination
    
    input:
        n - total number
        k - number of chosen
    
    output:
        c - number of combination
    '''
    return factorial(n) / factorial(k) / factorial(n - k)

In [4]:
def neg(loss,i,prod):
    '''
    Compute negative function and gradient information
    
    input:
        loss - loss function
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fnt - negative function value
        gfnt - negative function gradient
    '''
    fnt = 0.0 # n stands for negative
    gfnt = 0.0
    for k in range(i,N+1):
        # compute forward difference
        delta = 0.0
        for j in range(k+1):
            delta += (-1)**(k-j)*comb(k,j)*loss(j/N)
        # compute coefficient
        beta = comb(N,k)*comb(k,i)*(N+1)*delta/(2*L)**k
        # compute function value
        fnt += beta*(L/2-prod)**(k-i)
        # compute gradient
        gfnt += beta*(k-i)*(L/2-prod)**(k-i-1)  # no xt yet!
    return fnt,gfnt

In [5]:
def p_hat(t,yt,ptm1):
    '''
    Approximate probability
    
    input:
        t - iteration
        yt - label at t
        ptm1 - p at t-1
    
    output:
        pt - p at t
    '''
    pt = (t*ptm1 + (yt+1)/2)/(t+1) # m stands for minus
    return pt

In [6]:
def a_hat(t,fpt,yt,ptm1,atm1):
    '''
    Approximate primal a
    
    input:
        t - iteration
        fpt - positive function at t
        yt - sample label at t
        ptm1 - p at t-1
        atm1 - a at t-1
    
    output:
        at - a at t
    '''
    at = (fpt*((yt+1)/2) + t*ptm1*atm1)/(t+1) # do not update pt yet!
    return at

In [7]:
def b_hat(t,fmt,yt,ptm1,btm1):
    '''
    Approximate primal b
    
    input:
        t - iteration
        fmt - negative function at t
        yt - sample label at t
        ptm1 - p at t-1
        btm1 - b at t-1
    
    output:
        bt - b at t-1
    '''
    bt = (fmt*((-yt+1)/2) + t*(1-ptm1)*btm1)/(t+1) # indicator of y=-1!
    return bt

In [8]:
def alpha_step(t,at,bt):
    '''
    Compute dual alpha
    
    input:
        t - iteration
        at - a at t
        bt - b at t
        
    output:
        alphat - alpha at t
    '''
    alphat = at + bt
    return alphat

In [9]:
def w_grad(gfpt,gfnt,yt,at,bt,alphat):
    '''
    Gradient with respect to w
    
    input:
        fpt - positive function at t
        gfpt - positive function gradient at t
        fnt - negative function at t
        gfnt - negative function gradient at t
        yt - sample label at t
        pt - p at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        gradwt - gradient w.r.t. w at t
    '''
    gradwt = 0.0
    if yt == 1:
        gradwt = 2*(alphat - at)*gfpt
    else:
        gradwt = 2*(alphat - bt)*gfnt
    return gradwt

In [10]:
def proj(wt,R):
    '''
    Projection
    
    input:
        wt - w at t
        R - radius
        
    output:
        proj - projected wt
    '''
    norm = np.linalg.norm(wt)
    if norm > R:
        wt = wt/norm*R
    return wt

In [11]:
def a_grad(fpt,yt,at):
    '''
    Gradient with respect to a
    
    input:
        fpt - positive function at t
        yt - sample label at t
        pt - p at t
        at - a at t
    
    output:
        gradat - gradient w.r.t a at t
    '''
    gradat = 0.0 
    if yt == 1:
        gradat = 2*(at - fpt)
    else:
        gradat = 2*at
    return gradat

In [12]:
def b_grad(fnt,yt,bt):
    '''
    Gradient with respect to b
    
    input:
        fnt - negative function at t
        yt - sample label at t
        pt - p at t
        bt - b at t
    
    output:
        gradbt - gradient w.r.t b at t
    '''
    gradbt = 0.0 
    if yt == 1:
        gradbt = 2*bt
    else:
        gradbt = 2*(bt - fnt)
    return gradbt

In [13]:
def alpha_grad(fpt,fnt,yt,alphat):
    '''
    Gradient with respect to alpha
    '''
    gradalphat = 0.0
    if yt == 1:
        gradalphat = -2*(alphat - fpt)
    else:
        gradalphat = -2*(alphat - fnt)
    return gradalphat

In [14]:
def obj(pt,fpt,fnt,at,bt,alphat,yt):
    '''
    Compute objective function value
    
    input:
        t - iteration
        pt - 
        wt - 
    
    output:
        F - objective funciton value
    '''
    F = 0.0
    if yt == 1:
        F = (1-pt)*(-pt*alphat**2 + 2*alphat*fpt+(fpt-at)**2 - fpt**2)
    else:
        F = pt*(-(1-pt)*alphat**2 + 2*alphat*fnt+(fnt-bt)**2 - fnt**2)
    return F

In [15]:
def loader(filename):
    '''
    Data file loader
    
    input:
        filename - filename
    
    output:
        x - sample features
        y - sample labels
    '''
    # raw data
    L = []
    with open(filename,'r') as file:
        for line in csv.reader(file, delimiter = ' '):
            line[0] = '0:'+line[0]
            line.remove('')
            L.append(dict(i.split(':') for i in line))
    df = pd.DataFrame(L,dtype=float).fillna(0)
    X = df.iloc[:,1:].values
    Y = df.iloc[:,0].values
    # normalize
    norm = np.linalg.norm(X,axis=1)
    X = X/norm[:,None]
    return X,Y

In [28]:
def prox(eta,loss,wj,aj,bj,alphaj,bwt,bat,bbt,balphat,x,y):
    '''
    perform proximal guided gradient descent when receive an sample
    '''
    prod = np.inner(wj,x)
    fpt = np.zeros(N+1)
    gfpt = np.zeros(N+1)
    fnt = np.zeros(N+1)
    gfnt = np.zeros(N+1)
    aJ = np.zeros(N+1)
    bJ = np.zeros(N+1)
    alphaJ = np.zeros(N+1)
    gradwt = 0.0
    gradat = 0.0
    gradbt = 0.0
    for i in range(N+1):
        fpt[i],gfpt[i] = pos(i,prod)
        fnt[i],gfnt[i] = neg(loss,i,prod)
        gradwt += w_grad(gfpt[i],gfnt[i],y,aj[i],bj[i],alphaj[i]) # accumulate i
        gradat = a_grad(fpt[i],y,aj[i])
        gradbt = b_grad(fnt[i],y,bj[i])
        gradalphat = alpha_grad(fpt[i],fnt[i],y,alphaj[i])
        aJ[i] = aj[i] - eta*(gradat/(N+1)+gamma*(aj[i]-bat[i]))
        bJ[i] = bj[i] - eta*(gradbt/(N+1)+gamma*(bj[i]-bbt[i]))
        alphaJ[i] = alphaj[i] + eta*gradalphat/(N+1)
    wJ = wj - eta*(gradwt*x/(N+1)+ gamma*(wj - bwt))
    wJ = proj(wJ,1)
    aJ = proj(aJ,1)
    bJ = proj(bJ,1)
    alphaJ = proj(alphaJ,1)
    
    return wJ,aJ,bJ,alphaJ

In [29]:
def PGSPD(t,loss,X,Y,bwt,bat,bbt,balphat):
    '''
    Proximally Guided Stochastic Primal Dual Algorithm
    '''
    
    # initialize inner loop variables
    Wt = bwt+0.0
    At = bat+0.0
    Bt = bbt+0.0
    ALPHAt = balphat+0.0
    
    BWt = np.zeros(d)
    BAt = np.zeros(N+1)
    BBt = np.zeros(N+1)
    BALPHAt = np.zeros(N+1)
    
    ETAt = np.zeros(t).reshape((t,1))
    # inner loop update at j
    for j in range(1,t+1): 
        # step size
        ETAt[j-1] = 1/np.sqrt(T)/M # M is the bound for gradient
        BWt += ETAt[j-1]*Wt
        BAt += ETAt[j-1]*At
        BBt += ETAt[j-1]*Bt
        BALPHAt += ETAt[j-1]*ALPHAt
        # update inner loop variables
        Wt,At,Bt,ALPHAt = prox(ETAt[j-1],loss,Wt,At,Bt,ALPHAt,bwt,bat,bbt,balphat,X[j-1,:],Y[j-1])
        
    # update outer loop variables
    bwt = BWt/sum(ETAt)
    bat = BAT/sum(ETAt)
    bbt = BBt/sum(ETAt)
    balphat = BALPHAt/sum(ETAt)
    
    return bwt,bat,bbt,balphat

In [30]:
hinge = lambda x:max(0,1+L-2*L*x)
logistics = lambda x:np.log(1+np.exp(L-2*L*x))

L = 2
N = 10
M,_ = neg(hinge,0,0) # weak convexity parameter
rho = M*N*N
gamma = rho+1
M = rho/N
T = 200 # DO NOT make it longer than n!

FEATURES,LABELS = loader('diabetes')

# get dimensions of the data
n,d = FEATURES.shape
    
# initialize outer loop variables
BWT = np.zeros(d) # d is the dimension of the features
BAT = np.zeros(N+1)
BBT = np.zeros(N+1)
BALPHAT = np.zeros(N+1)

roc_auc = np.zeros(T)
for t in range(1,T+1):
    # sample a point
    index = np.random.randint(n, size=t)
    #start = (t*(t-1)//2)%n
    #end = (t*(t+1)//2)%n
    features = FEATURES[index,:]
    labels = LABELS[index]
    # update outer loop variables
    BWT,BAT,BBT,BALPHAT = PGSPD(t,hinge,features,labels,BWT,BAT,BBT,BALPHAT)
    fpr, tpr, _ = roc_curve(LABELS, np.dot(FEATURES,BWT))
    roc_auc[t-1] = auc(fpr, tpr)
    print('iteration: %d AUC: %f' %(t,roc_auc[t-1]))

iteration: 1 AUC: 0.500000
iteration: 2 AUC: 0.500000
iteration: 3 AUC: 0.605224
iteration: 4 AUC: 0.252806
iteration: 5 AUC: 0.241910
iteration: 6 AUC: 0.217291
iteration: 7 AUC: 0.248604
iteration: 8 AUC: 0.254276
iteration: 9 AUC: 0.228254
iteration: 10 AUC: 0.218209
iteration: 11 AUC: 0.230284
iteration: 12 AUC: 0.211164
iteration: 13 AUC: 0.196537
iteration: 14 AUC: 0.201224
iteration: 15 AUC: 0.204194
iteration: 16 AUC: 0.208634
iteration: 17 AUC: 0.206448
iteration: 18 AUC: 0.202007
iteration: 19 AUC: 0.201164
iteration: 20 AUC: 0.206179
iteration: 21 AUC: 0.202515
iteration: 22 AUC: 0.195634
iteration: 23 AUC: 0.199104
iteration: 24 AUC: 0.199164
iteration: 25 AUC: 0.201269
iteration: 26 AUC: 0.205530
iteration: 27 AUC: 0.201216
iteration: 28 AUC: 0.201552
iteration: 29 AUC: 0.209291
iteration: 30 AUC: 0.214821
iteration: 31 AUC: 0.224276
iteration: 32 AUC: 0.233358
iteration: 33 AUC: 0.223619
iteration: 34 AUC: 0.216649
iteration: 35 AUC: 0.209761
iteration: 36 AUC: 0.207403
i

In [15]:
def SOLAM(t,batch,loss,pt,wt,at,bt,alphat):
    '''
    Stochastic Online AUC Maximization step
    
    input:
        T - total number of iteration
        F - objective function value
        loss - loss function
        pt - p at t
        wt - w at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        W - record of each wt
        A - record of each at
        B - record of each bt
        ALPHA - record of each alphat
    '''
    # Loop in the batch
    eta = 1/np.sqrt(t+1)/2
    for k in range(batch):
        
        # Update pt
        pt = p_hat(t*batch+k,y[(t*batch+k)%M],pt)
        # Update wt,at,bt
        prod = np.inner(wt,x[(t*batch+k)%M])
        fpt = np.zeros(N+1)
        gfpt = np.zeros(N+1)
        fnt = np.zeros(N+1)
        gfnt = np.zeros(N+1)
        gradwt = 0.0
        gradat = 0.0
        gradbt = 0.0
        
        
        for i in range(N+1): # add up info of each i
            fpt[i],gfpt[i] = pos(i,t,prod) # partial info
            fnt[i],gfnt[i] = neg(loss,i,t,prod)
            gradwt += w_grad(fpt[i],gfpt[i],fnt[i],gfnt[i],y[(t*batch+k)%M],pt,at[i],bt[i],alphat[i])
            gradat = a_grad(fpt[i],y[(t*batch+k)%M],pt,at[i])
            gradbt = b_grad(fnt[i],y[(t*batch+k)%M],pt,bt[i])
            at[i] -= eta*gradat/(N+1)/batch
            bt[i] -= eta*gradbt/(N+1)/batch
            alphat[i] = at[i]+bt[i]
            #F += obj(pt,fpt[i],fnt[i],at,bt,alphat,y[(t*batch+k)%T])
        
        wt -= eta*gradwt*y[(t*batch+k)%M]*x[(t*batch+k)%M]/(N+1)/batch # step size as 1/t gradient descent
        
    wt = proj(wt,1)    
        
    return pt,wt,at,bt,alphat

In [ ]:
L = 2 # range
N = 10 # degree
T = 100 # iteration

hinge = lambda x:max(0,1+L-2*L*x)
logistics = lambda x:np.log(1+np.exp(L-2*L*x))

pt = 0.0
wt = np.zeros(d)
at = np.zeros(N+1)
bt = np.zeros(N+1)
alphat = np.zeros(N+1)

roc_auc = np.zeros(T)
batch = 1
for t in range(T):
    pt,wt,at,bt,alphat = SOLAM(t,batch,hinge,pt,wt,at,bt,alphat)
    fpr, tpr, _ = roc_curve(y, np.dot(x,wt))
    roc_auc[t] = auc(fpr, tpr)
    print('iteration: %d AUC: %f' %(t+1,roc_auc[t]))